In [1]:
import pyfesom2 as pf
import xarray as xr
import numpy as np
import sys


from scipy.io import loadmat
from scipy.interpolate import griddata

sys.path.append("/p/home/jusers/mueller29/juwels/jupyter_notebooks/implicit_filtering/")
from implicit_filter import CuPyFilter
import math
import dask
from dask import delayed
import dask.array as da
from dask.distributed import Client
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status'})

osgeo is not installed, conversion to Geo formats like Geotiff (fesom2GeoFormat) will not work.


In [2]:
n_cores = 30
mem_lim = str(int(100*np.floor(960/n_cores)))+'MB' #96GB total memory, set to MB (96000), divide by number of cores, then round to next 100
if 'client' in locals() or 'client' in globals():
    client.close()
client = Client(n_workers=n_cores, threads_per_worker=1,memory_limit=mem_lim)
client.restart()
client.amm.start() #automatic memory management
client

2024-02-01 16:28:20,840 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/p/project/chhb19/mueller29/miniconda3/envs/implicit_filter/lib/python3.9/site-packages/distributed/core.py", line 1564, in _connect
    comm = await connect(
  File "/p/project/chhb19/mueller29/miniconda3/envs/implicit_filter/lib/python3.9/site-packages/distributed/comm/core.py", line 377, in connect
    handshake = await comm.read()
  File "/p/project/chhb19/mueller29/miniconda3/envs/implicit_filter/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/p/project/chhb19/mueller29/miniconda3/envs/implicit_filter/lib/python3.9/site-packages/distributed/core.py", line 1674, in connect
    return connect_attemp

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/8787/status,
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/8787/status,Workers: 30
Total threads: 30,Total memory: 89.41 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46441,Workers: 30
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/8787/status,Total threads: 30
Started: Just now,Total memory: 89.41 GiB
Comm: tcp://127.0.0.1:43301,Total threads: 1
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/38649/status,Memory: 2.98 GiB
Nanny: tcp://127.0.0.1:34585,


In [3]:
reg_x = np.arange(-3.1,3.1,.1)
reg_y = np.arange(-3,3.1,.1)
X,Y = np.meshgrid(reg_x,reg_y)

In [4]:
depth = 100

In [5]:
mat_path = '/p/project/chhb19/mueller29/Nencioli_tracking/'+str(depth)+'m/python_readable/'
vars = ['lons','lats','rads','speeds','types']
for var in vars:
    mat=loadmat(mat_path+'eddy_'+var+'.mat')
    ex_stri='all_'+var+'=mat[\''+var+'\'].flatten()'
    exec(ex_stri)
    
mat = loadmat(mat_path+'eddy_days.mat')
all_days=mat['days']

In [ ]:
all_days[0]

In [32]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Distance in kilometers
    distance = R * c
    
    return distance

def get_dxdy(lon1, lon2, lat1, lat2):
    # Calculate distance and angle
    dist = haversine(lon1, lat1, lon2, lat2)
    ang = np.arctan2(lon2 - lon1, lat2 - lat1)
    
    # Calculate dx and dy
    dx = dist * np.cos(ang)
    dy = dist * np.sin(ang)
    
    return dx, dy

def get_points_within_radius(lon_in, lat_in, lon_target, lat_target, radius, fac):
    # Compute distances from all points to the target point
    distances = haversine(lon_in, lat_in, lon_target, lat_target)
    max_radius = radius*fac
    # Filter points within the maximum radius
    within_radius_indices = np.where(distances <= max_radius)[0]
    within_radius_lon = lon_in[within_radius_indices]
    within_radius_lat = lat_in[within_radius_indices]
    
    dx =  within_radius_lon - lon_target
    dy =  within_radius_lat - lat_target

    # Calculate differences between target point and filtered points
    dx_km, dy_km = np.vectorize(get_dxdy)(lon_target, within_radius_lon, lat_target, within_radius_lat)
    dx_rel = dx_km/radius
    dy_rel = dy_km/radius
    return within_radius_indices, dx, dy, dx_km, dy_km, dx_rel, dy_rel

def process_eddy(lon_target, lat_target, radius_target, lon_in, lat_in, data_in, fac, X, Y):
    within_radius_indices, _, _, _, _, dx_rel, dy_rel = get_points_within_radius(lon_in, lat_in, lon_target, lat_target, radius_target, fac)
    data_tmp = data_in[within_radius_indices]
    gridded_data = griddata((dx_rel.ravel(), dy_rel.ravel()), data_tmp.ravel(), (X, Y), method='linear', fill_value=np.nan)
    #nan_mask = np.isnan(gridded_data)
    #gridded_data[nan_mask]=0
    #count = (~nan_mask).astype(int)
    del within_radius_indices
    del data_in
    del data_tmp
    del dx_rel
    del dy_rel
    del X
    del Y
    return gridded_data
@delayed
def process_date(lon_target, lat_target, type_target, radius_target, lon_in, lat_in, data_in, fac, X, Y, ind, date):
    tmp_path='/p/scratch/chhb19/mueller29/composites/100m/'
    gridded_data = process_eddy(lon_target, lat_target, radius_target, lon_in, lat_in, data_in, fac, X, Y)
    date_str = np.datetime_as_string(date,unit='D').replace('-', '')
    filename = 'gridded_'+date_str+'_'+str(ind).zfill(7)+'.nc'
    # Create the Dataset
    dout = xr.Dataset(
        data_vars=dict(
            gridded_data=(['x', 'y'], gridded_data),
            cyc=(['ee'], [type_target]),
            lon=(['ee'], [lon_target]),
            lat=(['ee'], [lat_target]),
            rad=(['ee'], [radius_target]),
            date=(['ee'], [date]),
        ),
        coords=dict(
            X=(['x', 'y'], X),
            Y=(['x', 'y'], Y),
            num=('ee', [ii])
        ),
    )
    dout.to_netcdf(tmp_path+filename)
    del gridded_data
    del dout

In [7]:
# paths
data_path = '/p/scratch/chhb19/mueller29/AO_40_nopc/'
mesh_path = '/p/project/chhb19/meshes/AO_40/'

mesh=pf.load_mesh(mesh_path)
model_lons=mesh.x2
model_lats=mesh.y2
elements=mesh.elem.astype('int32')
lons_rot,lats_rot = pf.ut.scalar_g2r(-90,90,90,model_lons,model_lats)

/p/project/chhb19/meshes/AO_40/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /p/project/chhb19/meshes/AO_40/pickle_mesh_py3_fesom2


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import cmocean

In [ ]:
plt.plot(dx,dy,'.')

In [ ]:
plt.scatter(dx_km,dy_km,5,c=np.hypot(dx_sw,dy_sw),cmap=cm.viridis)
plt.colorbar

In [ ]:
plt.scatter(dx_rel,dy_rel,5,c=np.hypot(dx_sw,dy_sw),cmap=cm.viridis)
plt.colorbar

In [8]:
jf = CuPyFilter.load_from_file('/p/home/jusers/mueller29/juwels/jupyter_notebooks/implicit_filtering/filter_cash_AO_1km.npz')

In [9]:
dxm = 1 # Approximate resolution of the mesh in km
Kc = np.array([100]) * dxm
Kc = 2 * math.pi / Kc  # Wavenumbers
nr = len(Kc)
cyclic_length = 360  # in degrees; if not cyclic, take it larger than  zonal size
cyclic_length = cyclic_length * math.pi / 180 



In [10]:
fac=5

In [ ]:
pf.plot(mesh,data,box=[-180, 180, 80, 90],mapproj='np',levels=np.arange(-2,5.1,.1),cmap=cmocean.cm.thermal)

In [ ]:
pf.plot(mesh,data_coarse,box=[-180, 180, 80, 90],mapproj='np',levels=np.arange(-2,5.1,.1),cmap=cmocean.cm.thermal)

In [ ]:
pf.plot(mesh,data_in,box=[-180, 180, 80, 90],mapproj='np',levels=np.arange(-3,3.1,.1),cmap=cmocean.cm.balance)


In [ ]:
im=plt.imshow(test[0])
plt.colorbar(im)

In [ ]:
im=plt.imshow(test2[0])
plt.colorbar(im)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm


In [ ]:
plt.pcolormesh(X,Y,result_array.mean(axis=0).squeeze())

In [11]:
time_chunk = 1;
nod2_chunk = 11538465;
#nod2_chunk = 2900000;
temp=xr.open_mfdataset((data_path+'temp_100.fesom.*.nc'),chunks={'time':time_chunk,'nod2':nod2_chunk})['temp_100']


In [12]:
%%time
dd=0
ind=0
ind=0




lons=all_lons[dd].flatten()
lats=all_lats[dd].flatten()
rads=all_rads[dd].flatten()
types=all_types[dd].flatten()
speeds=all_speeds[dd].flatten()
date=all_days[dd]

ok_size = rads > 1.5
ok_speed = speeds > 0

lons=lons[ok_size&ok_speed]
lats=lats[ok_size&ok_speed]
rads=rads[ok_size&ok_speed]
types=types[ok_size&ok_speed]
speeds=speeds[ok_size&ok_speed]   


# Zero-pad the month and day components
date_string = '-'.join([str(date[0]), f'{date[1]:02}', f'{date[2]:02}'])
# Convert to datetime object
datetime_object = np.datetime64(date_string,'ns')

CPU times: user 884 µs, sys: 626 µs, total: 1.51 ms
Wall time: 815 µs


'20150101'

In [ ]:
#load and high-pass filter data
data = temp[dd,:].values
data_coarse = jf.compute(1, Kc[-1], data)
data_in = data-data_coarse


#persist the input data
data_in_dask_array = da.from_array(data_in, chunks='auto')
client.persist(data_in_dask_array)
lon_in_dask_array = da.from_array(lons_rot, chunks='auto')
client.persist(lon_in_dask_array)
lat_in_dask_array = da.from_array(lats_rot, chunks='auto')
client.persist(lat_in_dask_array)
datetime_dask_array = da.from_array(datetime_object)
client.persist(datetime_dask_array)

#make composites 
results = []
for ii in range(len(lons)):
    lon_eddy = lons[ii]
    lat_eddy = lats[ii]
    type_eddy = types[ii]
    rad_eddy = rads[ii]
    results.append(process_date(lon_eddy, lat_eddy, type_eddy, rad_eddy, lon_in_dask_array, lat_in_dask_array, data_in_dask_array, fac, X, Y, ind, datetime_dask_array))    
    ind +=1
result=dask.compute(results)

In [ ]:
len(all_lons)

In [ ]:
dd

In [ ]:
ind = 0
for dd in range(len(all_lons)):
    lons=all_lons[dd].flatten()
    lats=all_lats[dd].flatten()
    rads=all_rads[dd].flatten()
    types=all_types[dd].flatten()
    speeds=all_speeds[dd].flatten()
    date=all_days[dd]

    ok_size = rads > 1.5
    ok_speed = speeds > 0

    lons=lons[ok_size&ok_speed]
    lats=lats[ok_size&ok_speed]
    rads=rads[ok_size&ok_speed]
    types=types[ok_size&ok_speed]
    speeds=speeds[ok_size&ok_speed]   


    # Zero-pad the month and day components
    date_string = '-'.join([str(date[0]), f'{date[1]:02}', f'{date[2]:02}'])
    # Convert to datetime object
    datetime_object = np.datetime64(date_string,'ns')


    #load and high-pass filter data
    data = temp[dd,:].values
    data_coarse = jf.compute(1, Kc[-1], data)
    data_in = data-data_coarse


    #persist the input data
    data_in_dask_array = da.from_array(data_in, chunks='auto')
    client.persist(data_in_dask_array)
    lon_in_dask_array = da.from_array(lons_rot, chunks='auto')
    client.persist(lon_in_dask_array)
    lat_in_dask_array = da.from_array(lats_rot, chunks='auto')
    client.persist(lat_in_dask_array)
    datetime_dask_array = da.from_array(datetime_object)
    client.persist(datetime_dask_array)

    #make composites 
    results = []
    for ii in range(len(lons)):
        lon_eddy = lons[ii]
        lat_eddy = lats[ii]
        type_eddy = types[ii]
        rad_eddy = rads[ii]
        results.append(process_date(lon_eddy, lat_eddy, type_eddy, rad_eddy, lon_in_dask_array, lat_in_dask_array, data_in_dask_array, fac, X, Y, ind, datetime_dask_array))    
        ind +=1
    result=dask.compute(results)
    

In [ ]:
%%time
# Enable dask for parallel I/O
xr.set_options(keep_attrs=True)
# Open the dataset with adjusted chunk size
gridded = xr.open_mfdataset(
    '/p/scratch/chhb19/mueller29/composites/100m/*.nc', 
    combine='nested', 
    concat_dim='ee', 
    parallel=True, 
    chunks={'x': 'auto', 'y': 'auto'}  # Adjust chunk size for 'x' and 'y' dimensions
)
gridded.to_netcdf('/p/scratch/chhb19/mueller29/composites/100m/all2.nc')

In [ ]:
%%time
xr.set_options(keep_attrs=True)
gridded = xr.open_mfdataset('/p/scratch/chhb19/mueller29/composites/100m/*.nc' , combine='nested', concat_dim='ee',parallel=True)
gridded.to_netcdf('/p/scratch/chhb19/mueller29/composites/100m/all.nc')

In [ ]:
cyc = gridded.cyc.values

In [ ]:
CE=gridded['gridded_data'][cyc==1,:,:]

In [ ]:
AE=gridded['gridded_data'][cyc==-1,:,:]

In [ ]:
CE.mean(dim='ee').plot(cmap=cmocean.cm.balance,vmin=-.5,vmax=.5)

In [ ]:
AE.mean(dim='ee').plot(cmap=cmocean.cm.balance,vmin=-.5,vmax=.5)